In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm
import json

In [3]:
# SQL db creation
conn = sqlite3.connect('stocks_data/sp500_light.db')
c = conn.cursor()

# create tables
c.execute('CREATE TABLE IF NOT EXISTS dimension_table (`symbol` varchar(255) PRIMARY KEY, `name` varchar(255) NOT NULL, `sector` varchar(255) NOT NULL)')
c.execute('CREATE TABLE IF NOT EXISTS fact_table (`symbol` varchar(255) NOT NULL, `date` date NOT NULL, `open` float NOT NULL, `high` float NOT NULL, `low` float NOT NULL, `close` float NOT NULL, `adj_close` float NOT NULL, `volume` float NOT NULL, PRIMARY KEY (symbol, date))')

In [4]:
# source: https://github.com/c0redumb/yahoo_quote_download

# To make print working for Python2/3
from __future__ import print_function

# Use six to import urllib so it is working for Python2/3
from six.moves import urllib
# If you don't want to use six, please comment out the line above
# and use the line below instead (for Python3 only).
#import urllib.request, urllib.parse, urllib.error

import time
import pandas as pd

'''
Starting on May 2017, Yahoo financial has terminated its service on
the well used EOD data download without warning. This is confirmed
by Yahoo employee in forum posts.
Yahoo financial EOD data, however, still works on Yahoo financial pages.
These download links uses a "crumb" for authentication with a cookie "B".
This code is provided to obtain such matching cookie and crumb.
'''

# Build the cookie handler
cookier = urllib.request.HTTPCookieProcessor()
opener = urllib.request.build_opener(cookier)
urllib.request.install_opener(opener)

# Cookie and corresponding crumb
_cookie = None
_crumb = None

# Headers to fake a user agent
_headers={
    'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11'
}

def _get_cookie_crumb():
    '''
    This function perform a query and extract the matching cookie and crumb.
    '''

    # Perform a Yahoo financial lookup on SP500
    req = urllib.request.Request('https://finance.yahoo.com/quote/^GSPC', headers=_headers)
    f = urllib.request.urlopen(req)
    alines = f.read().decode('utf-8')

    # Extract the crumb from the response
    global _crumb
    cs = alines.find('CrumbStore')
    cr = alines.find('crumb', cs + 10)
    cl = alines.find(':', cr + 5)
    q1 = alines.find('"', cl + 1)
    q2 = alines.find('"', q1 + 1)
    crumb = alines[q1 + 1:q2]
    _crumb = crumb

    # Extract the cookie from cookiejar
    global cookier, _cookie
    for c in cookier.cookiejar:
        if c.domain != '.yahoo.com':
            continue
        if c.name != 'B':
            continue
        _cookie = c.value

    # Print the cookie and crumb
    #print('Cookie:', _cookie)
    #print('Crumb:', _crumb)

def load_yahoo_quote(ticker, begindate, enddate, info = 'quote', format_output = 'list'):
    '''
    This function load the corresponding history/divident/split from Yahoo.
    '''
    # Check to make sure that the cookie and crumb has been loaded
    global _cookie, _crumb
    if _cookie == None or _crumb == None:
        _get_cookie_crumb()

    # Prepare the parameters and the URL
    tb = time.mktime((int(begindate[0:4]), int(begindate[4:6]), int(begindate[6:8]), 4, 0, 0, 0, 0, 0))
    te = time.mktime((int(enddate[0:4]), int(enddate[4:6]), int(enddate[6:8]), 18, 0, 0, 0, 0, 0))

    param = dict()
    param['period1'] = int(tb)
    param['period2'] = int(te)
    param['interval'] = '1d'
    if info == 'quote':
        param['events'] = 'history'
    elif info == 'dividend':
        param['events'] = 'div'
    elif info == 'split':
        param['events'] = 'split'
    param['crumb'] = _crumb
    params = urllib.parse.urlencode(param)
    url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?{}'.format(ticker, params)
    #print(url)
    req = urllib.request.Request(url, headers=_headers)

    # Perform the query
    # There is no need to enter the cookie here, as it is automatically handled by opener
    f = urllib.request.urlopen(req)
    alines = f.read().decode('utf-8')
    #print(alines)
    if format_output == 'list':
        return alines.split('\n')

    if format_output == 'dataframe':
        nested_alines = [line.split(',') for line in alines.split('\n')[1:]]
        cols = alines.split('\n')[0].split(',')
        adf = pd.DataFrame.from_records(nested_alines[:-1], columns=cols)
        return adf

In [5]:
# iterate over tickers
for ticker in tqdm(['AAPL', 'AMZN', 'GS', 'DIS']):
    
    # load and format data
    try:
        rawdata = load_yahoo_quote(ticker, '19900101', '20190101')
    except:
        print(ticker)
        continue
    names = rawdata[0].lower().replace(' ', '_').split(',')
    data = pd.DataFrame(rawdata[1:-1])
    df = pd.DataFrame(data[0].str.split(',').tolist(), columns = names)
    df.insert(0, 'symbol', ticker)
    
    # persist
    df.to_sql('fact_table', con=conn, if_exists='append', index=False)
    conn.commit()
conn.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.44s/it]
